<a href="https://colab.research.google.com/github/natanrajch/DiploDatos/blob/main/automl/diplodatos_automl_Optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Utilización de automl framework: Optuna
Grupo 103 - DiploDatos 2021


##Introducción

En el presente notebook hemos realizado la búsqueda de parámetros óptimos para el problema de clasificación con el que hemos competido en la materia de la diplomatura Aprendizaje Supervisado. 

El problema en cuestión se puede encontrar en https://www.kaggle.com/c/diplodatos-travel-insurance-prediction-data/. Consiste en una clasificación binaria para intentar predecir si un usuario comprará o no un seguro de viajes, utilizando como features datos de edad, tipo de empleo, salario anual, nivel de educación, tamaño de la familia, si es viajero frecuente y si posee enfermedades crónicas.

Se utilizarán las funcionalidades del Framework Optuna para automatizar la búsqueda de los mejores hiperparámetros en un modelo de clasificación de Random Forest, que resultó ganador en la competencia Kaggle.

##Dependencies

Instalamos la librería principal, sin problemas en entorno linux

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import datasets
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

##Carga Dataset

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/practico/data/travel_insurance_prediction_train.csv')

In [ ]:
X = df.drop(['Customer', 'TravelInsurance'], axis=1)
y = df['TravelInsurance']
X['Employment Type'] = X['Employment Type'].apply(lambda row: 1 if row == 'Private Sector/Self Employed' else 0)
X['GraduateOrNot'] = X['GraduateOrNot'].apply(lambda row: 1 if row == 'Yes' else 0)
X['FrequentFlyer'] = X['FrequentFlyer'].apply(lambda row: 1 if row == 'Yes' else 0)
X['EverTravelledAbroad'] = X['EverTravelledAbroad'].apply(lambda row: 1 if row == 'Yes' else 0)
X.head(2)

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad
0,33,1,1,550000,6,0,0,0
1,28,1,1,800000,7,0,1,0


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.10)

##Implementación de búsqueda mediante Optuna

Se debe implementar una función objetivo, que toma como parámetro a un trial.
La API de Optuna presenta 2 conceptos principales: study y trial. El study es una búsqueda de mejores hiperparámetros, dentro de la cual, cada intento es un trial.

La función objective tomará como parámetro un trial particular, y sugerirá mediante las funciones trial.suggest un conjunto de hiperparámetros conviniente mediante la optimización bayesiana.

Luego la función objective debe retornar un valor objetivo, que en este caso lo elegimos como el promedio del cross validation score con un cv=3

Se utiliza también la función de la API de guardar un study de modo local, para poder continuar con el mismo más adelante

In [ ]:
from sklearn.model_selection import cross_val_score
def objective(trial):
      n_estimators = trial.suggest_int('n_estimators', 2, 20)
      max_depth = int(trial.suggest_loguniform('max_depth', 1, 32))
      criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
      max_samples = trial.suggest_int('max_samples', 1, 894)
      clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion, max_samples=max_samples)
      return cross_val_score(clf, X_train, y_train, 
           n_jobs=-1, cv=3).mean()

In [32]:
import logging
import sys
import optuna

# Se utiliza un stream handler para guardar el study en entorno local
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "travel_insurance_prediction"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

#Se crea el study y se corre el método de optimizar la función objective, durante 10000 trials
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='maximize')
study.optimize(objective, n_trials=10000, show_progress_bar=True)
#Aprox. 1 hs de funcionamiento en Colab

[I 2021-12-07 22:19:54,786] A new study created in memory with name: no-name-0e0eb8b8-a85e-41d2-a133-557c7dba90bd
/usr/local/lib/python3.7/dist-packages/optuna/progress_bar.py:47: ExperimentalWarning:

Progress bar is experimental (supported from v1.2.0). The interface can change in the future.



  0%|          | 0/10000 [00:00<?, ?it/s]

Se han truncado las últimas 5000 líneas del flujo de salida.
[I 2021-12-07 22:40:40,912] Trial 5000 finished with value: 0.8150633855331842 and parameters: {'n_estimators': 9, 'max_depth': 6.33134821245447, 'criterion': 'gini', 'max_samples': 894}. Best is trial 3584 with value: 0.8292319164802388.
[I 2021-12-07 22:40:41,257] Trial 5001 finished with value: 0.8165548098434005 and parameters: {'n_estimators': 8, 'max_depth': 6.979341047147129, 'criterion': 'entropy', 'max_samples': 853}. Best is trial 3584 with value: 0.8292319164802388.
[I 2021-12-07 22:40:41,812] Trial 5002 finished with value: 0.8180462341536168 and parameters: {'n_estimators': 18, 'max_depth': 5.532830436159586, 'criterion': 'entropy', 'max_samples': 825}. Best is trial 3584 with value: 0.8292319164802388.
[I 2021-12-07 22:40:42,203] Trial 5003 finished with value: 0.8202833706189411 and parameters: {'n_estimators': 17, 'max_depth': 5.740188656086642, 'criterion': 'gini', 'max_samples': 840}. Best is trial 3584 with

##Visualizaciones y análisis del estudio (study)

###Visualizaciones

In [38]:
optuna.visualization.plot_optimization_history(study)

In [39]:
optuna.visualization.plot_slice(study)

###Análisis del resultado

In [40]:
n_estimators = study.best_trial.params["n_estimators"]
max_depth = study.best_trial.params["max_depth"]
criterion = study.best_trial.params["criterion"]
max_samples = study.best_trial.params['max_samples']
clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, criterion=criterion, max_samples=max_samples)
clf.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=7.329480015199677,
                       max_samples=791, n_estimators=10)

In [41]:
from sklearn.metrics import f1_score
print('Training:', f1_score(y_train, clf.predict(X_train)), 'Validation:', f1_score(y_val, clf.predict(X_val)))

Training: 0.7162673392181589 Validation: 0.7710843373493975


##Comparación con datos de test
Se utiliza el dataset de test de la competencia, y se presenta un nuevo resultado a la misma (como Late Submission) para comparar con los resultados obtenidos sin Optuna

In [42]:
df_test = pd.read_csv('https://raw.githubusercontent.com/DiploDatos/AprendizajeSupervisado/master/practico/data/travel_insurance_prediction_test.csv')
X_test = df_test.drop(['Customer'], axis=1)
X_test['Employment Type'] = X_test['Employment Type'].apply(lambda row: 1 if row == 'Private Sector/Self Employed' else 0)
X_test['GraduateOrNot'] = X_test['GraduateOrNot'].apply(lambda row: 1 if row == 'Yes' else 0)
X_test['FrequentFlyer'] = X_test['FrequentFlyer'].apply(lambda row: 1 if row == 'Yes' else 0)
X_test['EverTravelledAbroad'] = X_test['EverTravelledAbroad'].apply(lambda row: 1 if row == 'Yes' else 0)

result = pd.DataFrame()
result['Customer'] = df_test.Customer
result['TravelInsurance'] = clf.predict(X_test)
result.to_csv('result_clf_optuna_grupo103.csv', index=False)

##Conclusiones

El modelo obtenido mediante la optimización de Optuna ha resultado superior a los modelos de random forest que se habían conseguido durante la competencia (comparado contra el grupo de uno de los integrantes del grupo 103), teniendo en cuenta el Public Score (es decir, datos de test no utilizados en entrenamiento ni validación), aunque no superior al valor que ganó la competencia.

Esto se podría mejorar indudablemente entregando más tiempo, y más capacidad de cómputo que el que es posible mediante un entorno Colab, y proponiendo un espacio de búsqueda aún mayor.